<a href="https://colab.research.google.com/github/varunsawhney8/Projects/blob/main/seleniumchrome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [9]:
import pandas as pd
import string
import re
from os import listdir
from nltk.corpus import stopwords
import numpy as np
import nltk
import requests
import os
#import streamlit as st
import altair as alt
#from newspaper import Article
from bs4 import BeautifulSoup as bs
import requests
#from newspaper import Config
from selenium import webdriver
import time
company={1: ['INFOSYS','INFY.NS'], 2:['TATA CONSULTANCY SERVICES','TCS.NS'] , 3: ['RELIANCE INDUSTRIES','RELIANCE.NS'],
 4: ['ICICI BANK','ICICIBANK.NS'] , 5: ['HDFC BANK','HDFCBANK.NS'], 6:[ 'HCL TECHNOLOGIES','HCLTECH.NS'],
 7:['BHARTI AIRTEL','BHARTIARTL.NS'], 8: ['INDUSIND BANK','INDUSINDBK.NS'], 9: ['STATE BANK OF INDIA','SBIN.NS'],
10: ['LARSEN & TOUBRO','LTI.NS'], 11: ['TECH MAHINDRA','TECHM.NS'], 12: ['AXIS BANK','AXISBANK.NS'],
13: ['ITC','ITC.NS'], 14: ['BAJAJ AUTO','BAJAJ-AUTO.NS'], 15:['OIL AND NATURAL GAS CORPORATION','ONGC.NS'],
16: ['TATA STEEL','TATASTEEL.NS'], 17: ['NTPC','NTPC.NS'], 18: ['MAHINDRA & MAHINDRA','M&M.NS'],
19: ['ASIAN PAINTS','ASIANPAINT.NS'], 20:[ 'POWER GRID CORPORATION OF INDIA','POWERGRID.NS'],
21: ['BAJAJ FINSERV','BAJAJFINSV.NS'], 22: ['TITAN COMPANY','TITAN.NS'],23: ['NESTLE INDIA','NESTLEIND.NS'],
24:['ULTRATECH CEMENT','ULTRACEMCO.NS'], 25: ['SUN PHARMACEUTICAL INDUSTRIES','SUNPHARMA.NS'],
26:['BAJAJ FINANCE','BAJFINANCE.NS'], 27: ['MARUTI SUZUKI INDIA','MARUTI.NS'], 28: ['HOUSING DEVELOPMENT FINANCE CORP.','HDFC.NS'],
29: [ 'HINDUSTAN UNILEVER','HINDUNILVR.NS'], 30: ['KOTAK MAHINDRA BANK','KOTAKBANK.NS'] }

x=input("Enter any company code: ")
x=int(x)
url = "https://www.ndtv.com/search?searchtext="+str(f"{company[x][0]}")

browser.get(url)

for i in range(0,1):
    browser.find_element_by_xpath("//a[@class= 'src_mre-btn']").click()
    time.sleep(10)

browser.find_element_by_xpath("//a[@class='notnow']").click()
time.sleep(5)

for i in range(0,7):
    browser.find_element_by_xpath("//a[@class= 'src_mre-btn']").click()
    time.sleep(10)

soup = bs(browser.page_source,"html.parser")
news = soup.find_all("article", class_ = "col-650 ml-auto")

title_data = []  
title_content = []

for news_1 in news:
    news_message = news_1.find_all("div", class_ = "src_itm-ttl")
    for i in range(0,len(news_message)):
        title_content.append(news_message[i].get_text())
        title_content[:] = [titles.lstrip('\n') for titles in title_content]
        title_content[:] = [titles.rstrip('\n') for titles in title_content]
        title_data = title_data + title_content

date_data = []  
date_content = []

for date_1 in news:
    date_publish = date_1.find_all("span", class_ = "src_itm-stx")
    for i in range(0,len(date_publish)):
        date_content.append(date_publish[i].get_text())
        date_content[:] = [dates.lstrip('\n') for dates in date_content]
        date_content[:] = [dates.rstrip('\n') for dates in date_content]
        date_data = date_data + date_content

############ converting into dataframe 

ndtv_data = pd.DataFrame(columns = ["Date", "Title"])

ndtv_data["Title"] = title_data
ndtv_data["Date"] = date_data

ndtv_data.drop_duplicates(subset=None, inplace=True)

#### Changing date format

ndtv_data["Date"]= ndtv_data["Date"].str.replace('January', "01") \
.str.replace('February', "02") \
    .str.replace('March', "03,") \
        .str.replace('April', "04,") \
            .str.replace('May', "05,") \
                .str.replace('June', "06,") \
                    .str.replace('July', "07,") \
                        .str.replace('August', "08,") \
                            .str.replace('September', "09,") \
                                .str.replace('October', "10,") \
                                    .str.replace('November', "11,") \
                                        .str.replace('December', "12,")

ndtv_data['Date'] = ndtv_data['Date'].str[5:]


def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)

ndtv_data['Date']=ndtv_data['Date'].apply(lambda x: find_number(x))

ndtv_data['Date'] = ndtv_data['Date'].str[-10:]

ndtv_data['Date'] = ndtv_data['Date'].str.replace(r'\D', '')

ndtv_data["Date"] = pd.to_datetime(ndtv_data['Date'], format = '%m%d%Y')

#st.write(ndtv_data.head())
#st.write(ndtv_data.tail())

# Yahoo Finance Data

import yfinance as yf 

company_nse = yf.download(f'{company[x][1]}',start=ndtv_data['Date'][ndtv_data.index[-1]] ,end=ndtv_data['Date'][ndtv_data.index[0]])

################### Merging data ##############################################
## company
## Replacing column name with spaces into underscores

company_nse.columns = company_nse.columns.str.replace(" ", "_")

company_nse_2 =company_nse[["Close"]]

#st.write(company_nse_2.head())
#st.write(company_nse_2.tail())



Enter any company code: 1


ElementClickInterceptedException: ignored